In [83]:
###Reyes, Marcus Group 7

###CoE 197Z Project 1.1

import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization
from keras import regularizers
from keras.optimizers import adam,sgd
from keras import optimizers

import numpy as np
from numpy import genfromtxt

from sklearn import preprocessing

import matplotlib.pyplot as plt

import string

#from sixfunctions import load_train, load_x_test, clean_data_with_mean
!pip install category_encoders

In [0]:
from importlib import reload
import sixfunctions
sixfunctions = reload(sixfunctions)
from sixfunctions import load_train, load_x_test, preprocess_data

In [177]:
train   = pd.read_csv("train_set_values.csv")
labels  = pd.read_csv("train_set_labels.csv")
test    = pd.read_csv("test_values.csv")

columns = set(train.columns)
drop_cols = {'id', 'num_private',
             'quality_group',   # exact duplicate of quality
             'recorded_by'}     # uniform value
unique_cols = { 'funder',       # 1898 unique values
                'ward',         # 2092 unique values
                'installer',    # 2146 unique values
                'scheme_name',  # 2697 unique values
                'subvillage',   # 19288 unique values
                'wpt_name' }    # 37400 unique values
binary_cols = {"permit", "public_meeting"}
object_cols = set(train.select_dtypes(include=['object']).columns) - drop_cols
onehot_cols = ((((object_cols | {'region_code', 'district_code'})
                - {'date_recorded', 'permit', 'public_meeting'})
                - unique_cols) - drop_cols)
#print(f"onehot_cols = {str(onehot_cols)}")
scale_cols = set(train.select_dtypes(include=['number']).columns)   \
                - onehot_cols - unique_cols - binary_cols - drop_cols | {"year_recorded"}

train, labels = preprocess_data(train, labels, test, drop_cols,
                                unique_cols, binary_cols, onehot_cols, scale_cols)

dropping: {'num_private', 'recorded_by', 'id', 'quality_group'}
CLEANING  amount_tsh
CLEANING  construction_year
CLEANING  population
cyclic encoding:
	date_recorded -> (doy_recorded_sin, doy_recorded_cos)
Normalization:
	construction_year [(1960.0, 2013.0)] -> [(-3.6574406124580006, 1.6079677259760634)]
	gps_height [(-90, 2770)] -> [(-1.0940495369292693, 3.0322765453176466)]
	amount_tsh [(0.2, 350000.0)] -> [(-0.3590986010964154, 117.97090065790498)]
	latitude [(-11.64944018, -2e-08)] -> [(-2.0174538540773703, 1.9368783817775395)]
	population [(1.0, 30500.0)] -> [(-0.6199883131902885, 66.89122175181514)]
	year_recorded [(2002, 2013)] -> [(-10.348549589913485, 1.1247289744119464)]
	longitude [(0.0, 40.34519307)] -> [(-5.188894889753043, 0.9543790152566973)]
one-hot encoding:
	payment:	7 categories
	source_type:	7 categories
	quantity_group:	5 categories
	region:	21 categories
	payment_type:	7 categories
	water_quality:	8 categories
	basin:	9 categories
	source_class:	3 categories
	lga:

In [0]:
x = train.to_numpy()
y = labels.to_numpy()

#### Old

In [0]:
do_not_include = {'quality_group',      # exact duplicate of 'quality'
                  'recorded_by',        # uniform value

                  'wpt_name',           #unique
                  'subvillage',         #dup of loc, numerous
                  'scheme_name',        #messy numerous
                  'payment',            #duplicate of payment type
                  'quantity_group',     #Dup of quantity
                  'waterpoint_type_group',#dup of wtptype less data
                  'source_type',        #dup of source
                  'extraction_type',    #dup of extr_type_group
                  #'extraction_type_group',
                  'region'} - unique_cols#enc in regcode

#These are sort of ordinal data
do_not_one_hot = {'id','gps_height','doy_recorded_sin','doy_recorded_cos',
                  'longitude','latitude','population','amount_tsh'} | unique_cols

clean_up = {'population','amount_tsh'}

#Tentatively do not include.
do_not_include_tent = {'funder','installer','ward','lga'} - unique_cols

#Don't know what it means 
do_not_include_temp = {'num_private'} - unique_cols


x, train_col,y = load_train("train_set_values.csv","train_set_labels.csv",
                            do_not_include, do_not_one_hot,
                            clean_up, do_not_include_tent, do_not_include_temp,
                            unique_cols)

#### Model

In [179]:
###Model

hidden = 1024
dropout = 0.4

(trash, input_dim) = x.shape

activation = 'relu'

model = Sequential()


model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))

model.add(Dense(hidden, input_dim = input_dim))
model.add(Dropout(dropout))
model.add(Activation(activation))


model.add(Dense(3,input_dim = hidden))
model.add(Activation('softmax'))

#model.summary()

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#history = model.fit(x, y, epochs=20, batch_size=4096*16, validation_split=0.2)
history = model.fit(x, y, epochs=20, batch_size=256, validation_split=0.2)

Train on 47520 samples, validate on 11880 samples
Epoch 1/20
47520/47520 [==============================] - 6s 127us/step - loss: 0.6673 - acc: 0.7211 - val_loss: 0.5857 - val_acc: 0.7650
Epoch 2/20
47520/47520 [==============================] - 3s 67us/step - loss: 0.5702 - acc: 0.7644 - val_loss: 0.5509 - val_acc: 0.7770
Epoch 3/20
47520/47520 [==============================] - 3s 67us/step - loss: 0.5305 - acc: 0.7818 - val_loss: 0.5364 - val_acc: 0.7844
Epoch 4/20
47520/47520 [==============================] - 3s 67us/step - loss: 0.5028 - acc: 0.7931 - val_loss: 0.5272 - val_acc: 0.7851
Epoch 5/20
47520/47520 [==============================] - 3s 67us/step - loss: 0.4804 - acc: 0.8028 - val_loss: 0.5234 - val_acc: 0.7873
Epoch 6/20
47520/47520 [==============================] - 3s 68us/step - loss: 0.4592 - acc: 0.8106 - val_loss: 0.5177 - val_acc: 0.7928
Epoch 7/20
47520/47520 [==============================] - 3s 67us/step - loss: 0.4392 - acc: 0.8188 - val_loss: 0.5106 - val_ac

KeyboardInterrupt: ignored

In [57]:

max_score = -1
iter_of_max = 0,0
train_plot = []
val_plot = []
x_axis = []

#Save the weights for k-fold validation
model.save_weights('model.h5')

amount = 120
k_folds = 5

###When i use k_folds instead of 10 for the np.zeros initialization it acts up
val_plot = np.zeros((k_folds,amount))
train_plot = np.zeros((k_folds,amount))
test_plot = np.zeros((k_folds,amount)) 
x_axis = np.zeros((k_folds,amount))
total = int(x.shape[0])
for j in range(k_folds):

    whole = np.arange(0,total)
    test_range = np.arange((j)*(total/k_folds), (j)*(total/k_folds)+(total/k_folds),dtype = 'int')
    
    train_range = np.delete(whole, test_range)
    
    
    #K-fold validation setup
    x_train = x[train_range,:]
    x_pretest = x[test_range,:]
    y_train = y[train_range,:]
    y_pretest = y[test_range,:]
    
    for i in range(amount):

        history = model.fit(x_train, y_train, epochs = 1, batch_size = 4096*16, verbose = 0)
        score = model.evaluate(x_pretest, y_pretest, batch_size = 512, verbose = 0)

        if  float(100 * score[1]) > float(max_score):
            max_score = float(100 * score[1])
            iter_of_max = i,j
        if i%10 == 0:
            print("----------",i,"-",j,"-----------")
            print("Test accuracy: ", (100.0 * score[1]))
            print("Maxscore: ", max_score, "at", iter_of_max,"epoch-kthfold")
        
        #for the plot
        x_axis[j,i] = i
        test_plot[j,i] = (score[1])
        train_plot[j,i] = np.array(history.history['acc'])

        

    if j == k_folds - 1:
        break
    model.load_weights('model.h5')
    print("Reloading Model")
    
print("Done evaluating performance")

---------- 0 - 0 -----------
Test accuracy:  54.23400671393783
Maxscore:  54.23400671393783 at (0, 0) epoch-kthfold
---------- 10 - 0 -----------
Test accuracy:  67.4242424081873
Maxscore:  67.74410774009397 at (9, 0) epoch-kthfold
---------- 20 - 0 -----------
Test accuracy:  71.74242421834155
Maxscore:  71.74242421834155 at (20, 0) epoch-kthfold
---------- 30 - 0 -----------
Test accuracy:  73.41750840546707
Maxscore:  73.80471382478271 at (28, 0) epoch-kthfold
---------- 40 - 0 -----------
Test accuracy:  75.06734007536762
Maxscore:  75.06734007536762 at (40, 0) epoch-kthfold
---------- 50 - 0 -----------
Test accuracy:  75.90067340870095
Maxscore:  75.90067340870095 at (50, 0) epoch-kthfold
---------- 60 - 0 -----------
Test accuracy:  76.82659933462689
Maxscore:  76.95286196088952 at (59, 0) epoch-kthfold
---------- 70 - 0 -----------
Test accuracy:  77.66835018039151
Maxscore:  77.66835018039151 at (70, 0) epoch-kthfold
---------- 80 - 0 -----------
Test accuracy:  77.76094274087

KeyboardInterrupt: ignored